In [1]:
import os
import pandas as pd

# save the filepaths for the data
school_data_file = os.path.join("Resources", "schools_complete.csv")
students_data_file = os.path.join("Resources", "students_complete.csv")

In [2]:
# Load the data
students_df = pd.read_csv(students_data_file)
schools_df = pd.read_csv(school_data_file)
schools_df.columns

Index(['School ID', 'school_name', 'type', 'size', 'budget'], dtype='object')

In [3]:
# students_df['student_name'][3] has a prefix that should not be there, these are high school kids not Doctor's. I need to find and clean the names of improper prefixes and suffixes.
students_names = students_df['student_name'].tolist()
prefixes = list()
suffixes = list()

# iterate through the names looking for the ones that need fixed.
for index, name in enumerate(students_names):
    if len(name.split(" ")) >=3:    
        if len(name.split(' ')[0]) <= 4:  # save the prefixes of the names that need fixed
            prefixes.append(name.split(' ')[0]) 
        else:
            pass
        if len(name.split(' ')[-1]) <= 3:    # save the suffixes of the names that need fixed
            suffixes.append(name.split(' ')[-1])

# make the prefixes and suffixes lists contain only unique terms
prefixes = list(set(prefixes)) 
suffixes = list(set(suffixes))

# print(suffixes)  # used the terminal print to manually find the unwanted prefixes and suffixes amoung the list of names that were saved by the filter.
prefixes_suffixes = [' MD', ' PhD', ' DDS', ' DVM', 'Dr. ', 'Miss ', 'Mr. ', 'Mrs. ', 'Ms. ']

# find and fix the prefixes and suffixes, do not delete the names or family names that were saved by the filter.
for word in prefixes_suffixes:
    students_df['student_name'] = students_df['student_name'].str.replace(word, '')

students_df.head()

C:\Users\Nick\anaconda3\envs\PyCity\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: The default value of regex will change from True to False in a future version.


,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [4]:
# merge all the data into one dataframe
all_df = students_df.merge(schools_df, on='school_name')

# Gather data for the whole district and create a summary: , , % passed in subject, % passed both
budget = all_df['budget'].unique().sum()    # total budget
total_schools = len(all_df['school_name'].unique())    # total number of schools in the district
total_students = all_df['Student ID'].count()    # total students
math_mean = all_df['math_score'].mean()    # average test scores for math
reading_mean = all_df['reading_score'].mean()    # average test scores for reading
pass_math = all_df[all_df['math_score'] >= 70]['Student ID'].count()    # total students passing math
pass_reading = all_df[all_df['reading_score'] >= 70]['Student ID'].count()    # total students passing reading
pass_math_reading = all_df[(all_df['math_score'] >= 70) & (all_df['reading_score'] >= 70)]['Student ID'].count() # total students passing both subjects
math_passing_percent = pass_math / total_students*100    # percent of students passing math
reading_passing_percent = pass_reading / total_students*100    # percent of students passing reading 
pass_both_percent = pass_math_reading/total_students*100    # percent of students passing both subjects

district_summary_df = pd.DataFrame([{'Total Schools':total_schools, 'Total Budget':budget, 'Total Students': total_students, 'Average Math Score':math_mean, 'Average Reading Score':reading_mean, '% Passing Math':math_passing_percent, '% Passing Reading':reading_passing_percent, '% Overall Passing':pass_both_percent}])
# district_summary_df
all_df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


In [5]:

# format the District Summary DataFrame
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df['% Passing Math'] = district_summary_df["% Passing Math"].map("{:.1f}%".format)
district_summary_df['% Passing Reading'] = district_summary_df["% Passing Reading"].map("{:.1f}%".format)
district_summary_df['% Overall Passing'] = district_summary_df["% Overall Passing"].map("{:.1f}%".format)
district_summary_df['Average Math Score'] = district_summary_df['Average Math Score'].map("{:.1f}".format)
district_summary_df['Average Reading Score'] = district_summary_df['Average Reading Score'].map("{:.1f}".format)

district_summary_df

,Total Schools,Total Budget,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"$24,649,428.00","39,170",79.0,81.9,75.0%,85.8%,65.2%


In [6]:
# New Dataframe to be the summary of all the school information
school_summary_df = pd.DataFrame()

# School type with the school names as the index
school_summary_df['School Type']= schools_df.set_index(['school_name'])['type']

# Total students
school_summary_df['Total Students'] = all_df['school_name'].value_counts()

# Total Budget
school_summary_df['Budget'] = schools_df.set_index(['school_name'])['budget']

# Total Budget per Student
school_summary_df['Budget Per Student'] = school_summary_df['Budget']/school_summary_df['Total Students']

# Average math and reading scores 
per_school_mean = all_df.groupby(['school_name']).mean()
school_summary_df['Average Math Score'] = per_school_mean['math_score'] 
school_summary_df['Average Reading Score'] = per_school_mean['reading_score'] 

# % passing math
passing_math = all_df[all_df['math_score'] >= 70].groupby(['school_name']).count()
school_summary_df['% Passing Math'] = (passing_math['math_score']/school_summary_df['Total Students'])*100

# % passing reading
passing_reading = all_df[all_df['reading_score'] >= 70].groupby(['school_name']).count()
school_summary_df['% Passing Reading'] = (passing_reading['reading_score']/school_summary_df['Total Students'])*100

# % overall passing
overall_passing = all_df[(all_df['math_score'] >= 70) & (all_df['reading_score'] >= 70)].groupby(['school_name']).count()
school_summary_df['% Passing Overall'] = (overall_passing['math_score']/school_summary_df['Total Students'])*100

school_summary_df


,School Type,Total Students,Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,89.892107
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166


In [7]:

# Format the school summary table
school_summary_df["Total Students"] = school_summary_df["Total Students"].map("{:,}".format)
school_summary_df["Budget"] = school_summary_df["Budget"].map("${:,.2f}".format)
school_summary_df["Budget Per Student"] = school_summary_df["Budget Per Student"].map("${:,.2f}".format)
school_summary_df['% Passing Math'] = school_summary_df["% Passing Math"].map("{:.1f}%".format)
school_summary_df['% Passing Reading'] = school_summary_df["% Passing Reading"].map("{:.1f}%".format)
school_summary_df['% Passing Overall'] = school_summary_df["% Passing Overall"].map("{:.1f}%".format)
school_summary_df['Average Math Score'] = school_summary_df['Average Math Score'].map("{:.1f}".format)
school_summary_df['Average Reading Score'] = school_summary_df['Average Reading Score'].map("{:.1f}".format)
school_summary_df

,School Type,Total Students,Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6,81.2,65.7%,81.3%,53.5%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7,81.2,66.0%,80.7%,53.2%
Shelton High School,Charter,"1,761","$1,056,600.00",$600.00,83.4,83.7,93.9%,95.9%,89.9%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.3,80.9,66.8%,80.9%,53.5%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.4,83.8,93.4%,97.1%,90.6%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.3,84.0,93.9%,96.5%,90.6%
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.1,84.0,94.1%,97.0%,91.3%
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.0,81.0,66.7%,81.9%,54.6%
Holden High School,Charter,427,"$248,087.00",$581.00,83.8,83.8,92.5%,96.3%,89.2%


In [10]:
# find the top 5 Best performing schools
school_summary_df.sort_values(['% Passing Overall'], ascending=False).head(5)

,School Type,Total Students,Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.1,84.0,94.1%,97.0%,91.3%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.4,83.8,93.3%,97.3%,90.9%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.4,83.8,93.4%,97.1%,90.6%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.3,84.0,93.9%,96.5%,90.6%
Pena High School,Charter,962,"$585,858.00",$609.00,83.8,84.0,94.6%,95.9%,90.5%


In [11]:
# find the top 5 worst performing schools
school_summary_df.sort_values(['% Passing Overall'], ascending=True).head(5)

,School Type,Total Students,Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.6,81.2,65.7%,81.3%,53.5%
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.3,80.9,66.8%,80.9%,53.5%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.1,81.0,66.1%,81.2%,53.5%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.7,81.2,66.0%,80.7%,53.2%
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.8,80.7,66.4%,80.2%,53.0%
